In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
import ast
import pandas as pd
import openpyxl
import re 
import os
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [3]:
from parcer_functions import data_loader_tys, data_loader_percentages

In [5]:
years_list = ['2020']
dates2 = ['01'] #начало года
dates1 = ['12'] #конец года
property_ids = ["10", "30", "25", "40", "50", "60", "70",          # Основные показатели
"110", "111", "112",                                # высоколиквидные активы
"200", "210", "220", "230", "240", "250", "260",    # кредиты ФЛ
"120", "121",                                       # выданные мбк
"300", "310", "320", "330", "340", "350", "360",    # кредиты ЮрЛ
"130", "140",                                       # вложения в акции, облигации
"190",                                              # бумаги переданные в репо
"150", "160", "170", "180",                         # вложения в вексел, др орг, капексб прочие
"400", "410", "420", "430", "440", "450", "460",    # Вклады физических лиц
"500", "510", "520", "530", "540", "550", "560",    # Средства предприятий и организаций
"600", "620",                                       # Привлеченные МБК
"700", "710", "720",                                # Выпущенные облигации и векселя
"1900"]                                             #ЛОРО-счета

In [6]:
data_tys = data_loader_tys(years_list, dates2, dates1, property_ids)

  0%|          | 0/1 [00:00<?, ?it/s]

IndexError: list index out of range

In [8]:
prop_id = '10'
data1 = '2020-01-01'
data2 = '2020-12-01'


In [14]:
url = f'https://www.banki.ru/banks/ratings/export.php?SEARCH_NAME=&SEARCH_REGN=&search[type]=name&sort_param=rating&sort_order=ASC&PROPERTY_ID={prop_id}&REGION_ID=0&date1={data1}&date2={data2}&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0' 
#выгружаем инфу из url
# response = requests.get(url)

# x = BeautifulSoup(response.content, 'html.parser')

In [4]:
url= 'https://www.banki.ru/banks/ratings/?PROPERTY_ID=120'

In [5]:
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)

In [6]:
driver.get(url) #запрос к странице


In [7]:
bs = BeautifulSoup(driver.page_source)

In [20]:
driver.find_element(by=By.NAME, value="my-text")

NameError: name 'By' is not defined

In [16]:
url = 'https://www.banki.ru/banks/ratings/?PROPERTY_ID=1600'

In [17]:
response = requests.get(url)

x = BeautifulSoup(response.content, 'html.parser')

In [171]:
##################################
def url_creator():
    for index, row in total_df1.iterrows():
        print(row['bank_name'], row['id'])
        pass

def extract_bank_table_from_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    #таблица 1
    table1 = soup.find_all('table', {'class' : 'standard-table'})[0]
    df = pd.read_html(str(table1))[0]
    df = df.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_тыс_кги', 'янв_тыс_руб', 'изм', 'процент'], axis=1)
    df = df[['Показатель', 'янв_тыс_руб']]
    df['янв_тыс_руб'] = df['янв_тыс_руб'].apply(lambda x: x.replace(' ', '')).astype(float)
    df = df.pivot_table(columns = 'Показатель', values='янв_тыс_руб')
    df_1 = df.reset_index().drop(columns='index')
    df_1 = df_1.drop(columns = ['Активы нетто','Вклады физических лиц','Капитал (по форме 123)','Чистая прибыль'])
    
    #таблица 2
    table2 = soup.find_all('table', {'class' : 'standard-table'})[1]
    df = pd.read_html(str(table2))[0]
    df = df.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_тыс_кги', 'янв_тыс_руб', 'изм', 'процент'], axis=1) #table1 - абсолютные показатели
    df = df[['Показатель', 'янв_тыс_руб']]
    df['янв_тыс_руб'] = df['янв_тыс_руб'].apply(lambda x: x.replace(' ', '')).astype(float)
    df = df.pivot_table(columns = 'Показатель', values='янв_тыс_руб')
    df_2 = df.reset_index().drop(columns='index')

    #таблица 3
    table3 = soup.find_all('table', {'class' : 'standard-table'})[2]
    df = pd.read_html(str(table3))[0]
    df = df.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_процент', 'янв_процент', 'изм', 'процент'], axis=1) #table2 - относительные показатели
    df = df[['Показатель', 'янв_процент']]
    df['янв_процент'] = df['янв_процент'].astype(float) / 100
    df = df.pivot_table(columns = 'Показатель', values='янв_процент')
    df_3 = df.reset_index().drop(columns='index')

    merged_row = pd.concat([df_1, df_2, df_3], axis = 1)

    return merged_row



In [ ]:
{'Активы нетто',
 'Вклады физических лиц',
 'Капитал (по форме 123)',
 'Чистая прибыль'}

In [142]:
df1.drop(columns = ['Активы нетто','Вклады физических лиц','Капитал (по форме 123)','Чистая прибыль'])

Показатель,Вложения в ценные бумаги,Кредитный портфель,Просроченная задолженность в кредитном портфеле
0,5.178299e+09,2.286929e+10,793445449.0


In [36]:
url = 'https://www.banki.ru/banks/ratings/?BANK_ID=322&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01'
response = requests.get(url)

soup = BeautifulSoup(response.content, 'html.parser')

In [37]:
soup.title.text    #имя банка  

'Сбербанк - рейтинг на основании показателей деятельности за период c 2021-01-01 по 2021-12-01 | Банки.ру'

In [42]:
table1 = soup.find_all('table', {'class' : 'standard-table'})[0]
df2 = pd.read_html(str(table1))[0]

In [43]:
df2_0 = df2.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_тыс_кги', 'янв_тыс_руб', 'изм', 'процент'], axis=1)

In [51]:
df2_1 = df2_0[['Показатель', 'янв_тыс_руб']]

In [59]:
df2_1['янв_тыс_руб'] = df2_1['янв_тыс_руб'].apply(lambda x: x.replace(' ', '')).astype(float)

In [62]:
df2_2 = df2_1.pivot_table(columns = 'Показатель', values='янв_тыс_руб')

In [63]:
df2_2

Показатель,Активы нетто,Вклады физических лиц,Вложения в ценные бумаги,Капитал (по форме 123),Кредитный портфель,Просроченная задолженность в кредитном портфеле,Чистая прибыль
янв_тыс_руб,3.482355e+10,1.480089e+10,5.178299e+09,4.741066e+09,2.286929e+10,793445449.0,781587558.0


In [189]:
table2 = soup.find_all('table', {'class' : 'standard-table'})[1]
df = pd.read_html(str(table2))[0]
df = df.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_тыс_кги', 'янв_тыс_руб', 'изм', 'процент'], axis=1) #table1 - абсолютные показатели
# df1 = df[['Показатель', 'янв_тыс_руб']]
# df1['янв_тыс_руб'] = df1['янв_тыс_руб'].apply(lambda x: x.replace(' ', '')).astype(float)
# df2 = df1.pivot_table(columns = 'Показатель', values='янв_тыс_руб')

In [190]:
df

,место_по_России,место_в_регионе,Показатель,дек_тыс_кги,янв_тыс_руб,изм,процент
0,1,1,Активы нетто,38 812 535 226,34 823 546 983,+3 988 988 243,"+11,45%"
1,1,1,Высоколиквидные активы,2 467 907 764,1 888 277 933,+579 629 831,"+30,70%"
2,1,1,Денежные средства в кассе,613 843 125,616 013 327,-2 170 202,"-0,35%"
3,1,1,Денежные средства в кассе оборот,8 580 860 761,11 011 038 986,-2 430 178 225,"-22,07%"
4,1,1,НОСТРО-счета,611 014 137,448 433 196,+162 580 941,"+36,26%"
...,...,...,...,...,...,...,...
63,1,1,Облигации,913 448 644,733 647 608,+179 801 036,"+24,51%"
64,1 +3,1 +2,Векселя,70 863 331,54 164 546,+16 698 785,"+30,83%"
65,1,1,Капитал (по форме 123),5 071 037 128,4 741 066 308,+329 970 820,"+6,96%"
66,1,1,ЛОРО-счета,284 583 860,111 447 342,+173 136 518,"+155,35%"


In [112]:
table3 = soup.find_all('table', {'class' : 'standard-table'})[2]
df = pd.read_html(str(table3))[0]
df = df.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_процент', 'янв_процент', 'изм', 'процент'], axis=1) #table2 - относительные показатели
df = df[['Показатель', 'янв_процент']]
df['янв_процент'] = df['янв_процент'].astype(float) / 100
df1 = df.pivot_table(columns = 'Показатель', values='янв_процент', index=None)

In [119]:
df1 = df1.reset_index().drop(columns='index')

In [121]:
df1.index.names

FrozenList([None])

In [122]:
df1

Показатель,Н1,Н2,Н3,Рентабельность активов-нетто,Рентабельность капитала,Уровень обеспечения кредитного портфеля залогом имущества,Уровень просроченной задолженности по кредитному портфелю,Уровень резервирования по кредитному портфелю
0,14.76,98.38,127.59,2.46,16.85,111.12,3.47,6.62


In [111]:
df1.reset_index()

Показатель,index,Н1,Н2,Н3,Рентабельность активов-нетто,Рентабельность капитала,Уровень обеспечения кредитного портфеля залогом имущества,Уровень просроченной задолженности по кредитному портфелю,Уровень резервирования по кредитному портфелю
0,янв_процент,14.76,98.38,127.59,2.46,16.85,111.12,3.47,6.62


In [172]:
url = 'https://www.banki.ru/banks/ratings/?BANK_ID=322&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01'
bnk_row = extract_bank_table_from_url(url)

In [175]:
bnk_row.to_excel('df2.xlsx', index=False)

In [147]:
df1

Показатель,Вложения в ценные бумаги,Кредитный портфель,Просроченная задолженность в кредитном портфеле
0,5.178299e+09,2.286929e+10,793445449.0


In [148]:
df2

Показатель,Активы нетто,Бумаги переданные в РЕПО,Векселя,Вклады физических лиц,Вклады физических лиц оборот,Вложения в акции,Вложения в векселя,Вложения в капиталы других организаций,Вложения в облигации,Выданные МБК,...,Сроком до 90 дней оборот,Сроком от 1 года до 3 лет,Сроком от 1 года до 3 лет оборот,Сроком от 181 дня до 1 года,Сроком от 181 дня до 1 года оборот,Сроком от 91 до 180 дней,Сроком от 91 до 180 дней оборот,Счета,Счета оборот,Чистая прибыль
0,3.482355e+10,850396291.0,54164546.0,1.480089e+10,1.469827e+10,38968241.0,936432.0,1.039507e+09,5.138394e+09,1.276263e+09,...,2.773673e+09,1.746018e+09,393670687.5,1.622587e+09,454771601.5,367235191.5,52927967.0,4.101529e+09,2.324612e+10,781587558.0


In [149]:
df3

Показатель,Н1,Н2,Н3,Рентабельность активов-нетто,Рентабельность капитала,Уровень обеспечения кредитного портфеля залогом имущества,Уровень просроченной задолженности по кредитному портфелю,Уровень резервирования по кредитному портфелю
0,14.76,98.38,127.59,2.46,16.85,111.12,3.47,6.62


In [159]:
y = pd.concat([df1, df2, df3], axis = 1)

In [158]:
x = total_df1[:1]

In [169]:
y['bank_name'] = 'Сбербанк'

In [170]:
y

Показатель,Вложения в ценные бумаги,Кредитный портфель,Просроченная задолженность в кредитном портфеле,Активы нетто,Бумаги переданные в РЕПО,Векселя,Вклады физических лиц,Вклады физических лиц оборот,Вложения в акции,Вложения в векселя,...,Чистая прибыль,Н1,Н2,Н3,Рентабельность активов-нетто,Рентабельность капитала,Уровень обеспечения кредитного портфеля залогом имущества,Уровень просроченной задолженности по кредитному портфелю,Уровень резервирования по кредитному портфелю,bank_name
0,5.178299e+09,2.286929e+10,793445449.0,3.482355e+10,850396291.0,54164546.0,1.480089e+10,1.469827e+10,38968241.0,936432.0,...,781587558.0,14.76,98.38,127.59,2.46,16.85,111.12,3.47,6.62,Сбербанк


In [168]:
total_df1['bank_name'].iloc[0]

'Сбербанк'

In [49]:
ids_dct = {'Сбербанк': '322',
           'ВТБ': '327'}

In [52]:
bank_id = ids_dct['ВТБ']

In [56]:
url = f'https://www.banki.ru/banks/ratings/?BANK_ID={bank_id}&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01'
response = requests.get(url)
x = BeautifulSoup(response.content, 'html.parser')

In [58]:
table1 = x.find_all('table', {'class' : 'standard-table'})[1]
df = pd.read_html(str(table1))[0]

In [ ]:
#находим все айдишники банков

In [60]:
page = 1
url = 'https://www.banki.ru/banks/ratings/?source=submenu_banksratings&PAGEN_1={page}'
response = requests.get(url)
x = BeautifulSoup(response.content, 'html.parser')

In [75]:
bank_id =x.find_all('a', {'class': 'widget__link'})[0].get('href')
bank_id

'?BANK_ID=322&PAGEN_1=0'

In [103]:
# match = re.search(r'\d{1,}', bank_id)

In [101]:
pattern = re.compile(r'\d{1,}')
match = re.findall(pattern, bank_id)

In [109]:
match[0]

'322'

In [173]:
def page_getter():
    for p in range(8):
        page = p + 1
        url = f'https://www.banki.ru/banks/ratings/?source=submenu_banksratings&PAGEN_1={page}'
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        yield soup

def table_creator_page(soup) -> pd.DataFrame:
    bank_id_table = pd.DataFrame(columns = ['bank_name', 'id'])
    names_and_ids = soup.find_all('a', {'class': 'widget__link'})
    for row, i in enumerate(names_and_ids):
        bank_id = i.get('href')
        pattern = re.compile(r'\d{1,}')
        match = re.findall(pattern, bank_id)
        bank_id_table.loc[row] = [i.text, match[0]]
    return bank_id_table

def total_banks_ids_dict_creator() -> pd.DataFrame:
    total_table = pd.DataFrame(columns = ['bank_name', 'id'])
    for soup in page_getter():
        time.sleep(3)
        page_bank_id_table = table_creator_page(soup)
        total_table = pd.concat([total_table, page_bank_id_table])
        print('page_dowloaded')
    return total_table




In [115]:
def f(r=8):
    for p in range(r):
        yield p + 1

In [163]:
for i in f():
    time.sleep(1)
    print(i)

1
2
3
4
5
6
7
8


In [118]:
a = f()

In [125]:
x = page_getter()

In [126]:
x1 = next(x)

In [131]:
name_ids = x1.find_all('a', {'class': 'widget__link'})

In [156]:
# for i in name_ids:
#     bank_id = i.get('href')
#     pattern = re.compile(r'\d{1,}')
#     match = re.findall(pattern, bank_id)
#     print(i.text, match[0])

In [157]:
tbl = table_creator_page(x1)

In [161]:
df_merged = pd.concat([tbl, tbl])

In [162]:
df_merged

,bank_name,id
0,Сбербанк,322
1,ВТБ,327
2,Газпромбанк,2764
3,Альфа-Банк,325
4,НКЦ (Национальный клиринговый центр),191161
...,...,...
45,Банк Синара (СКБ-банк),96003
46,Инвестторгбанк,10275
47,«Кубань Кредит»,191174
48,Локо-Банк,9183


In [174]:
total_df = total_banks_ids_dict_creator()

page_dowloaded
page_dowloaded
page_dowloaded
page_dowloaded
page_dowloaded
page_dowloaded
page_dowloaded
page_dowloaded


In [175]:
total_df['bank_name'].nunique()

353

In [179]:
total_df.to_excel('bank_ids.xlsx', index=False)

In [176]:
total_df['bank_name'].value_counts()

bank_name
Сбербанк                        1
Банк «Венец»                    1
Крокус-Банк                     1
Кетовский Коммерческий Банк     1
Уралфинанс                      1
                               ..
банк Раунд                      1
НБД-Банк                        1
Банк «Национальный стандарт»    1
«Хлынов»                        1
«ПЕРЕСВЕТ»                      1
Name: count, Length: 353, dtype: int64

In [3]:
from parcer_functions import Download_bank_ids

In [5]:
total_df1 = Download_bank_ids().total_banks_ids_dict_creator()

page_dowloaded
page_dowloaded
page_dowloaded
page_dowloaded
page_dowloaded
page_dowloaded
page_dowloaded
page_dowloaded


In [176]:
total_df1

,bank_name,id
0,Сбербанк,322
1,ВТБ,327
2,Газпромбанк,2764
3,Альфа-Банк,325
4,НКЦ (Национальный клиринговый центр),191161
...,...,...
48,«ИНЭКО»,5011664
49,ВБРР,16995
0,ПСБ,5306
1,СМП Банк,17200


In [41]:
for index, row in total_df1.iterrows():
    print(row['bank_name'], row['id'])

Сбербанк 322
ВТБ 327
Газпромбанк 2764
Альфа-Банк 325
НКЦ (Национальный клиринговый центр) 191161
Московский кредитный банк (МКБ) 7292
Россельхозбанк 4725
Банк «Открытие» 690
Совкомбанк 76620
Банк ДОМ.РФ 2237
Росбанк 3697
Райффайзен Банк 4389
Тинькофф Банк 195706
Национальный расчетный депозитарий (НРД) 242159
Банк «РОССИЯ» 68683
ЮниКредит Банк 4045
Банк «Санкт-Петербург» 68665
Ак Барс Банк 36119
Новикомбанк 9789
Уралсиб 63520
Почта Банк 194275
Ситибанк 8368
БМ-Банк 3425
АКБ «БЭНК ОФ ЧАЙНА» 187059
МТС Банк 6055
РНКБ 13011
АйСиБиСи Банк 384111
Уральский банк реконструкции и развития (УБРиР) 95970
ОТП Банк 16196
Банк ТКБ 11518
Банк ЗЕНИТ 863
Драйв Клик Банк 77476
Хоум Банк 6764
Росэксимбанк 8814
Абсолют Банк 9259
Точка 10728683
«Траст» 68195
Экспобанк 2631
ББР Банк 12092
Русский Стандарт 351
Азиатско-Тихоокеанский банк (АТБ) 192833
Кредит Европа Банк 41643
Металлинвестбанк 9011
Дж. П. Морган Банк Интернешнл 9700
Банк «Аверс» 193185
Банк Синара (СКБ-банк) 96003
Инвестторгбанк 10275
«Кубань

In [251]:
url = 'https://www.banki.ru/banks/ratings/?BANK_ID=322&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01'
bnk_row = extract_bank_table_from_url(url=url)

TypeError: extract_bank_table_from_url() missing 1 required positional argument: 'bank_name'

In [184]:
def url_generator(bank_ids_table, year):
    for _ , row in bank_ids_table.iterrows():
        row_id = row['id']
        bank_name = row['bank_name']
        url = f'https://www.banki.ru/banks/ratings/?BANK_ID={row_id}&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1={year}-12-01&date2={year}-01-01'
        yield bank_name, url

In [180]:
gn = url_generator(total_df1, 2021)

In [185]:
for i in url_generator(total_df1, 2021):
    print(i)

('Сбербанк', 'https://www.banki.ru/banks/ratings/?BANK_ID=322&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01')
('ВТБ', 'https://www.banki.ru/banks/ratings/?BANK_ID=327&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01')
('Газпромбанк', 'https://www.banki.ru/banks/ratings/?BANK_ID=2764&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01')
('Альфа-Банк', 'https://www.banki.ru/banks/ratings/?BANK_ID=325&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01')
('НКЦ (Национальный клиринговый центр)', 'https://www.banki.ru/banks/ratings/?BANK_ID=191161&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01')
('Московский кредитный банк (МКБ)', 'https://www.banki.ru/banks/ratings/?BANK_ID=7292&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01')
('Россельхозбанк', 'https://www.banki.ru/banks/ratings/?BANK_ID=4725&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date

In [187]:
from parcer_functions import extract_bank_table_from_url

In [188]:
extract_bank_table_from_url('Банк «Санкт-Петербург»', 'https://www.banki.ru/banks/ratings/?BANK_ID=68665&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01')

Показатель,Вложения в ценные бумаги,Кредитный портфель,Просроченная задолженность в кредитном портфеле,Активы нетто,Бумаги переданные в РЕПО,Векселя,Вклады физических лиц,Вклады физических лиц оборот,Вложения в акции,Вложения в векселя,...,Чистая прибыль,Н1,Н2,Н3,Рентабельность активов-нетто,Рентабельность капитала,Уровень обеспечения кредитного портфеля залогом имущества,Уровень просроченной задолженности по кредитному портфелю,Уровень резервирования по кредитному портфелю,bank_name
0,81898122.0,471225739.0,22047072.0,771966247.0,10358101.0,8056594.0,253322367.0,183978703.0,1117746.0,282878.0,...,11712465.0,13.5,114.41,134.74,1.62,13.89,93.72,4.68,11.39,Банк «Санкт-Петербург»


In [307]:
from parcer_functions import Download_bank_pages

In [308]:
ids_table = total_df1[:4]

In [309]:
ids_table

,bank_name,id
0,Сбербанк,322
1,ВТБ,327
2,Газпромбанк,2764
3,Альфа-Банк,325


In [310]:
df = Download_bank_pages(year = 2021, bank_ids_table=ids_table).parce_year_table()

In [314]:
df.to_excel('total_df_4.xlsx')

In [317]:
df.columns.name = 0
df = df.reset_index().drop(columns = 'index')

In [318]:
df

,Вложения в ценные бумаги,Кредитный портфель,Просроченная задолженность в кредитном портфеле,Активы нетто,Бумаги переданные в РЕПО,Векселя,Вклады физических лиц,Вклады физических лиц оборот,Вложения в акции,Вложения в векселя,...,ЮЛ Счета оборот,Н1,Н2,Н3,Рентабельность активов-нетто,Рентабельность капитала,Уровень обеспечения кредитного портфеля залогом имущества,Уровень просроченной задолженности по кредитному портфелю,Уровень резервирования по кредитному портфелю,bank_name
0,5.173248e+09,2.645656e+10,659923468.0,3.881254e+10,1.758385e+09,70863331.0,1.468500e+10,1.316737e+10,35086449.0,968513.0,...,3.354470e+10,14.18,113.11,112.05,3.50,25.76,108.73,2.45,4.60,Сбербанк
1,3.066817e+09,1.176611e+10,354047414.0,1.953169e+10,2.235843e+08,40935758.0,4.848598e+09,4.560879e+09,131401478.0,86549.0,...,2.492400e+10,11.25,92.63,93.25,1.43,14.86,33.96,2.90,5.52,ВТБ
2,7.049199e+08,5.556576e+09,108113191.0,8.479539e+09,6.649800e+04,22559675.0,1.446794e+09,7.921405e+08,18343172.0,0.0,...,1.561082e+10,12.04,68.27,104.47,1.68,16.24,88.34,1.91,3.72,Газпромбанк
3,5.100551e+08,3.750167e+09,139447600.0,5.749491e+09,0.000000e+00,13648885.0,1.656068e+09,1.759084e+09,27878883.0,961.0,...,8.256204e+09,14.21,92.78,109.99,2.71,20.82,51.97,3.66,5.39,Альфа-Банк


In [239]:
df = df.reset_index().drop(columns = 'index')

In [248]:
len(df.columns)

60

In [267]:
ids_table

,bank_name,id
0,Сбербанк,322
1,ВТБ,327
2,Газпромбанк,2764
3,Альфа-Банк,325


In [280]:
df_1, df_2, df_3 = extract_bank_table_from_url('Сбербанк', 'https://www.banki.ru/banks/ratings/?BANK_ID=2764&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01')

In [279]:
# df_2.head(50)

In [281]:
df_2 = df_2.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_тыс_руб', 'янв_тыс_руб', 'изм', 'процент'], axis=1)

In [313]:
df_2.to_excel('df_2.xlsx')

In [319]:
############################

In [326]:
total_df1[5:6]

,bank_name,id
5,Московский кредитный банк (МКБ),7292


In [332]:
total_df1

,bank_name,id
0,Сбербанк,322
1,ВТБ,327
2,Газпромбанк,2764
3,Альфа-Банк,325
4,НКЦ (Национальный клиринговый центр),191161
...,...,...
48,«ИНЭКО»,5011664
49,ВБРР,16995
0,ПСБ,5306
1,СМП Банк,17200


In [350]:
total_df2 = total_df1.drop(index=4).drop(index=13)

In [371]:
total_df2

,bank_name,id
0,Сбербанк,322
1,ВТБ,327
2,Газпромбанк,2764
3,Альфа-Банк,325
5,Московский кредитный банк (МКБ),7292
...,...,...
48,«ИНЭКО»,5011664
49,ВБРР,16995
0,ПСБ,5306
1,СМП Банк,17200


In [443]:
from parcer_functions import Download_bank_pages
cls_parcer = Download_bank_pages(year = 2021, bank_ids_table=total_df2[68:])

In [444]:
cls_parcer.parce_year_table()

-----
«Коммерческий Индо Банк» https://www.banki.ru/banks/ratings/?BANK_ID=17237&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
-----
bank row downloaded 1
bank row downloaded 2
bank row downloaded 3
-----
Дойче Банк https://www.banki.ru/banks/ratings/?BANK_ID=8487&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
-----
bank row downloaded 4
bank row downloaded 5
bank row downloaded 6
-----
Эм-Ю-Эф-Джи Банк (Евразия) https://www.banki.ru/banks/ratings/?BANK_ID=191159&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
-----
bank row downloaded 7
-----
Мир Бизнес Банк https://www.banki.ru/banks/ratings/?BANK_ID=152898&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
-----
bank row downloaded 8
bank row downloaded 9
-----
Коммерцбанк (Евразия) https://www.banki.ru/banks/ratings/?BANK_ID=9179&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
-----
-----
«КЭБ ЭйчЭнБи Банк» https:

In [445]:
current_df = cls_parcer.final_df

In [446]:
cur_df_for_save = cls_parcer.prettify_final_df(current_df)
cur_df_for_save

,bank_name,Вложения в ценные бумаги,Кредитный портфель,Просроченная задолженность в кредитном портфеле,Активы нетто,Бумаги переданные в РЕПО,Векселя,Вклады физических лиц,Вклады физических лиц оборот,Вложения в акции,...,ЮЛ Счета Сроком от 91 до 180 дней оборот,ЮЛ Счета оборот,Н1,Н2,Н3,Рентабельность активов-нетто,Рентабельность капитала,Уровень обеспечения кредитного портфеля залогом имущества,Уровень просроченной задолженности по кредитному портфелю,Уровень резервирования по кредитному портфелю
0,Фора-Банк,1940495.0,41548556.0,980461.0,6.200566e+07,0.0,143412.0,31602214.0,15851084.0,805649.0,...,27622.0,5.761219e+07,13.32,66.08,78.05,1.67,11.05,78.48,2.21,8.59
1,КИВИ Банк,11746589.0,3497705.0,373026.0,4.474527e+07,0.0,0.0,65772.0,31051.0,0.0,...,0.0,9.946669e+07,18.80,29.76,113.52,20.25,60.63,0.41,10.56,10.98
2,Банк «Приморье»,13849459.0,19179338.0,2724865.0,4.349564e+07,0.0,19881.0,21472148.0,11185330.0,149068.0,...,336224.0,4.414943e+07,11.56,68.69,70.57,1.10,13.33,72.39,14.60,14.77
3,ВУЗ-банк,22111450.0,136625656.0,1157573.0,1.667923e+08,20742379.0,0.0,11791416.0,1924427.0,23146.0,...,7870.0,6.721589e+06,0.00,149.94,58.43,0.13,NaN,59.98,0.83,2.51
4,Банк «Объединенный капитал»,33225859.0,1942422.0,20149.0,6.272558e+07,0.0,735207.0,32872658.0,6302476.0,0.0,...,32000.0,2.096904e+07,26.89,48.16,94.56,1.18,8.70,293.59,0.93,11.57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,«Альтернатива»,NaN,0.0,0.0,3.831590e+05,0.0,0.0,51.0,0.0,0.0,...,0.0,0.000000e+00,49.52,0.00,0.00,NaN,NaN,NaN,NaN,NaN
207,Эко-Инвест,NaN,10702.0,5176.0,1.028070e+06,0.0,0.0,665925.0,335254.0,0.0,...,0.0,7.886100e+04,71.32,NaN,144.84,-0.43,-1.35,104.40,44.99,51.94
208,ВБРР,154778136.0,517708197.0,2275396.0,1.080275e+09,0.0,2063883.0,136486058.0,67932275.0,136530.0,...,37756880.0,7.454662e+09,17.32,69.83,74.75,1.15,8.11,47.28,0.43,1.65
209,СМП Банк,90952424.0,211379106.0,9239525.0,6.803226e+08,4161395.0,10416586.0,260611096.0,67746523.0,156012.0,...,1039030.0,4.177431e+08,13.18,86.96,121.56,0.29,3.28,70.54,4.30,5.52


In [447]:
cur_df_for_save.to_excel('chasti/df8.xlsx', index = False)

In [442]:
total_df2[68:90]

,bank_name,id
22,«Коммерческий Индо Банк»,17237
23,Фора-Банк,80493
24,КИВИ Банк,189185
25,Банк «Приморье»,191082
26,Дойче Банк,8487
27,ВУЗ-банк,99562
28,Банк «Объединенный капитал»,68806
29,Челиндбанк,191044
30,Эм-Ю-Эф-Джи Банк (Евразия),191159
31,Модульбанк,195649


In [ ]:
#проблемы с :
#БМ-Банк 2
# Росэксимбанк 
# Точка 
# Дж. П. Морган Банк Интернешнл 
# Сумитомо Мицуи Рус Банк
# СДМ-Банк
# МСП Банк

In [440]:
for i in [1,2,3,[['4']],5,6]:
    try:
         p = i + 1
    except:
         print('smth_wrong')
         continue
    print(p)

2
3
4
smth_wrong
6
7


In [450]:
df1 = pd.read_excel('chasti/df1.xlsx')
df2 = pd.read_excel('chasti/df2.xlsx')
# df1 = pd.read_excel('chasti/df3.xlsx')
# df1 = pd.read_excel('chasti/df4.xlsx')
# df1 = pd.read_excel('chasti/df5.xlsx')
# df1 = pd.read_excel('chasti/df6.xlsx')
# df1 = pd.read_excel('chasti/df7.xlsx')
# df1 = pd.read_excel('chasti/df8.xlsx')



In [454]:
df1 = pd.read_excel('chasti/df1.xlsx')
for i in ['chasti/df2.xlsx','chasti/df3.xlsx','chasti/df4.xlsx','chasti/df5.xlsx','chasti/df6.xlsx','chasti/df7.xlsx','chasti/df8.xlsx']:
    df_i = pd.read_excel(i)
    df1 = pd.concat([df1, df_i])


In [457]:
df1.to_excel('chasti/data_2021.xlsx', index=False)

In [458]:
# ###########################################################################################################

In [459]:
# def extract_bank_table_from_url(self, bank_name, url):    #показатели на конец года
url = 'https://www.banki.ru/banks/ratings/?BANK_ID=17237&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01'

response = requests.get(url)
time.sleep(2)
soup = BeautifulSoup(response.content, 'html.parser')

#таблица 1
table1 = soup.find_all('table', {'class' : 'standard-table'})[0]   #ключевые показатели


In [463]:
df = pd.read_html(str(table1))[0]
df = df.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_тыс_руб', 'янв_тыс_руб', 'изм', 'процент'], axis=1)

In [464]:
df

,место_по_России,место_в_регионе,Показатель,дек_тыс_руб,янв_тыс_руб,изм,процент
0,212 -13,124 -4,Активы нетто,5 602 668,8 139 455,-2 536 787,"-31,17%"
1,197 +12,112 +9,Чистая прибыль,53 406,47 928,+5 478,"+11,43%"
2,163 +6,112 +4,Капитал (по форме 123),2 247 617,2 142 787,+104 830,"+4,89%"
3,260 +38,132 +17,Кредитный портфель,608 938,501 365,+107 573,"+21,46%"
4,302 +32,142 +21,Просроченная задолженность в кредитном портфеле,436,333,+103,"+30,93%"
5,282 -146,145 -63,Вложения в ценные бумаги,0,н/д,NaN,н/д


In [465]:
df = df[['Показатель', 'дек_тыс_руб']]
df['дек_тыс_руб'] = df['дек_тыс_руб'].apply(lambda x: x.replace(' ', '')).astype(float)
df = df.pivot_table(columns = 'Показатель', values='дек_тыс_руб')
df_1 = df.reset_index().drop(columns='index')

In [470]:
df_1

Показатель,Активы нетто,Вложения в ценные бумаги,Капитал (по форме 123),Кредитный портфель,Просроченная задолженность в кредитном портфеле,Чистая прибыль
0,5602668.0,0.0,2247617.0,608938.0,436.0,53406.0


In [472]:
df_1.drop(columns = ['Активы нетто','Капитал (по форме 123)','Чистая прибыль'])

Показатель,Вложения в ценные бумаги,Кредитный портфель,Просроченная задолженность в кредитном портфеле
0,0.0,608938.0,436.0


In [468]:
table2 = soup.find_all('table', {'class' : 'standard-table'})[1] #абсолютные показатели
df = pd.read_html(str(table2))[0]
df = df.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_тыс_руб', 'янв_тыс_руб', 'изм', 'процент'], axis=1) 
df = df[['Показатель', 'дек_тыс_руб']]
df['дек_тыс_руб'] = df['дек_тыс_руб'].apply(lambda x: x.replace(' ', '')).astype(float)

In [469]:
df

,Показатель,дек_тыс_руб
0,Активы нетто,5602668.0
1,Высоколиквидные активы,1421793.0
2,Денежные средства в кассе,17949.0
3,Денежные средства в кассе оборот,0.0
4,НОСТРО-счета,1402453.0
...,...,...
63,Облигации,0.0
64,Векселя,0.0
65,Капитал (по форме 123),2247617.0
66,ЛОРО-счета,187579.0


In [485]:
total_df3 = total_df1[total_df1['bank_name'].isin(['НКЦ (Национальный клиринговый центр)',
                                                   'Национальный расчетный депозитарий (НРД)'
                                                    '«Коммерческий Индо Банк»', 
                                                    'Дойче Банк' ,
                                                    'ЭмЮЭфДжи Банк (Евразия)' ,
                                                    'Мир Бизнес Банк' ,
                                                    'Коммерцбанк (Евразия)' ,
                                                    '«КЭБ ЭйчЭнБи Банк»' ,
                                                    'Ozon Банк' ,
                                                    'БМВ Банк' ,
                                                    'Тимер Банк', 
                                                    'Платежный Центр', 
                                                    'Российская Финансовая Корпорация' ,
                                                    'Банк «Держава»' ,
                                                    'АзияИнвест Банк' ,
                                                    '«Мобильная карта»' ,
                                                    'Еврофинанс Моснарбанк' ,
                                                    'ЭйчЭсБиСи Банк (HSBC)' ,
                                                    '«Фольксваген Банк РУС»' ,
                                                    'ИНКАХРАН' ,
                                                    'Яндекс Банк' ,
                                                    'Чайнасельхозбанк' ,
                                                    'Креди Агриколь КИБ' ,
                                                    'ЮMoney' ,
                                                    'БЖФ Банк' ,
                                                    'ЦМРБанк' ,
                                                    'СПБ Клиринг' ,
                                                    'РДК' ,
                                                    '«Натиксис Банк»' ,
                                                    'Голдман Сакс Банк' ,
                                                    '«Расчетные Решения»' ,
                                                    'Юнистрим' ,
                                                    '«ИС Банк»' ,
                                                    'Элекснет' ,
                                                    'Ю Би Эс Банк' ,
                                                    'Банк 131' ,
                                                    'Первый Клиентский Банк' ,
                                                    'РНКО «Единая касса»', 
                                                    'СанктПетербургский Банк Инвестиций' ,
                                                    'Вестерн Юнион ДП Восток' ,
                                                    '«Капитал»' ,
                                                    'Америкэн Экспресс Банк' ,
                                                    'НКО «Платежи и Расчеты»' ,
                                                    'Банк Стрела' ,
                                                    'РусьРегионБанк' ,
                                                    'Мурманский расчетный центр' ,
                                                    'Межбанковский Кредитный Союз' ,
                                                    'Платежный Конструктор', 
                                                    'Банк «Континенталь»' ,
                                                    'Северстройбанк' ,
                                                    '«БАНК БЕРЕЙТ»' ,
                                                    'ЮСиЭс' ,
                                                    '«Тайдон»' ,
                                                    '«ЭЛЕКСИР»' ,
                                                    'Истина' ,
                                                    'Нода' ,
                                                    'АЗИЯПЭЙ' ,
                                                    'Промсвязьинвест' ,
                                                    '«Петербургский Расчетный Центр»' ,
                                                    '«Дальний Восток»' ,
                                                    '«ИНЭКО»' ,
                                                    'ПСБ' ])]
total_df3.shape

(56, 2)

In [486]:
from parcer_functions import Download_bank_pages
cls_parcer = Download_bank_pages(year = 2021, no_fl_assets= True, bank_ids_table=total_df3)

In [487]:
cls_parcer.parce_year_table()

bank row downloaded 1
bank row downloaded 2
Мир Бизнес Банк https://www.banki.ru/banks/ratings/?BANK_ID=152898&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
bank row downloaded 3
bank row downloaded 4
Ozon Банк https://www.banki.ru/banks/ratings/?BANK_ID=11752949&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
bank row downloaded 5
Тимер Банк https://www.banki.ru/banks/ratings/?BANK_ID=192887&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
bank row downloaded 6
Российская Финансовая Корпорация https://www.banki.ru/banks/ratings/?BANK_ID=191004&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
Банк «Держава» https://www.banki.ru/banks/ratings/?BANK_ID=9751&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
bank row downloaded 7
Еврофинанс Моснарбанк https://www.banki.ru/banks/ratings/?BANK_ID=4374&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2021-12-01&date2=2021-01-01
ban

In [489]:
current_df = cls_parcer.final_df

In [491]:
cur_df_for_save = cls_parcer.prettify_final_df(current_df)

In [492]:
df_2021 = pd.read_excel('chasti/data_2021.xlsx')

In [497]:
df_2021_full = pd.concat([df_2021, cur_df_for_save])

In [498]:
# cur_df_for_save

In [500]:
df_2021_full.to_excel('full_dfs/df_2021_full.xlsx', index=False)

In [501]:
dct = {}

In [502]:
dct['проблемный банк'] = 'some_url'

In [503]:
dct

{'проблемный банк': 'some_url'}

In [139]:
url = 'https://www.banki.ru/banks/ratings/?BANK_ID=327&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2023-12-01&date2=2023-01-01'
response = requests.get(url)
time.sleep(2)
soup = BeautifulSoup(response.content, 'html.parser')

In [140]:
#таблица 1
table1 = soup.find_all('table', {'class' : 'standard-table'})[0]   #ключевые показатели
df = pd.read_html(str(table1))[0]
df = df.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_тыс_руб', 'янв_тыс_руб', 'изм', 'процент'], axis=1)
df = df[['Показатель', 'дек_тыс_руб']]
df['дек_тыс_руб'] = df['дек_тыс_руб'].apply(lambda x: x.replace(' ', '')).astype(float)
df = df.pivot_table(columns = 'Показатель', values='дек_тыс_руб')
df_1 = df.reset_index().drop(columns='index')

In [141]:
df_1

Показатель,Активы нетто,Вклады физических лиц,Вложения в ценные бумаги,Капитал (по форме 123),Кредитный портфель,Просроченная задолженность в кредитном портфеле,Чистая прибыль
0,2.618822e+10,4.579261e+09,4.693619e+09,1.708020e+09,1.573751e+10,135018281.0,185532759.0


In [145]:
#таблица 2
table2 = soup.find_all('table', {'class' : 'standard-table'})[1] #абсолютные показатели
df = pd.read_html(str(table2))[0]
df = df.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_тыс_руб', 'янв_тыс_руб', 'изм', 'процент'], axis=1) 
df = df[['Показатель', 'дек_тыс_руб']]
df['дек_тыс_руб'] = df['дек_тыс_руб'].apply(lambda x: x.replace(' ', '')).astype(float)
df.iloc[15, df.columns.get_loc('Показатель')] = 'ФЛ Счета'

In [147]:
df = df.pivot_table(columns = 'Показатель', values='дек_тыс_руб')

In [149]:
table3 = soup.find_all('table', {'class' : 'standard-table'})[2]    #относительные показатели
df = pd.read_html(str(table3))[0]
df = df.set_axis(['место_по_России', 'место_в_регионе','Показатель', 'дек_процент', 'янв_процент', 'изм', 'процент'], axis=1) 

In [151]:
df = df[['Показатель', 'дек_процент']]

In [152]:
df

,Показатель,дек_процент
0,Уровень просроченной задолженности по кредитно...,86
1,Н1,946
2,Н2,6091
3,Н3,8050


In [153]:
df['дек_процент'] = df['дек_процент'].astype(float) / 100
df = df.pivot_table(columns = 'Показатель', values='дек_процент')

In [154]:
df

Показатель,Н1,Н2,Н3,Уровень просроченной задолженности по кредитному портфелю
дек_процент,9.46,60.91,80.5,0.86


In [ ]:
# #####################################################################2020

In [110]:
ids_df = pd.read_excel('full_dfs/bank_ids.xlsx')

In [160]:
ids_df[:5]

,bank_name,id
0,Сбербанк,322
1,ВТБ,327
2,Газпромбанк,2764
3,Альфа-Банк,325
4,НКЦ (Национальный клиринговый центр),191161


In [161]:
from parcer_functions import Download_bank_pages
cls_parcer = Download_bank_pages(year = 2023, bank_ids_table=ids_df)

In [162]:
cls_parcer.parce_year_table()

bank row downloaded 1
bank row downloaded 2
bank row downloaded 3
bank row downloaded 4
НКЦ (Национальный клиринговый центр) https://www.banki.ru/banks/ratings/?BANK_ID=191161&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2023-12-01&date2=2023-01-01
bank row downloaded 5
bank row downloaded 6
bank row downloaded 7
bank row downloaded 8
bank row downloaded 9
bank row downloaded 10
bank row downloaded 11
bank row downloaded 12
Национальный расчетный депозитарий (НРД) https://www.banki.ru/banks/ratings/?BANK_ID=242159&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2023-12-01&date2=2023-01-01
bank row downloaded 13
bank row downloaded 14
bank row downloaded 15
bank row downloaded 16
bank row downloaded 17
bank row downloaded 18
bank row downloaded 19
bank row downloaded 20
bank row downloaded 21
bank row downloaded 22
bank row downloaded 23
bank row downloaded 24
АйСиБиСи Банк https://www.banki.ru/banks/ratings/?BANK_ID=384111&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2023-12-01&date2=2023-01

In [163]:
fin_df = cls_parcer.final_df

In [ ]:
fin_df.columns.name = 0
fin_df = fin_df.reset_index().drop(columns = 'index')
df_for_save = fin_df[['bank_name','Вложения в ценные бумаги', 'Кредитный портфель',
       'Просроченная задолженность в кредитном портфеле', 'Активы нетто',
       'Векселя', 'Вклады физических лиц', 'Вложения в акции',
       'Вложения в векселя', 'Вложения в капиталы других организаций',
       'Вложения в облигации', 'Выданные МБК',
       'Выпущенные облигации и векселя', 'Высоколиквидные активы',
       'Капитал (по форме 123)', 'Кредиты предприятиям и организациям',
       'Кредиты физическим лицам', 'ЛОРО-счета', 'НОСТРО-счета', 'Облигации',
       'Основные средства и нематериальные активы', 'Привлеченные МБК',
       'Привлеченные от ЦБ РФ', 'Прочие активы', 'Размещенные МБК в ЦБ РФ',
       'Средства предприятий и организаций', 'ФЛ Счета', 'Чистая прибыль',
       'Н1', 'Н2', 'Н3',
       'Уровень просроченной задолженности по кредитному портфелю']]

In [165]:
# df_for_save = cls_parcer.prettify_final_df(fin_df)

In [173]:
df_for_save.to_excel('chasti/data_2023.xlsx', index=False)

In [174]:
problem_banks_list = cls_parcer.problem_banks

In [175]:
problem_ids_df = ids_df[ids_df['bank_name'].isin(problem_banks_list)]
problem_ids_df

,bank_name,id
4,НКЦ (Национальный клиринговый центр),191161
13,Национальный расчетный депозитарий (НРД),242159
26,АйСиБиСи Банк,384111
31,Драйв Клик Банк,77476
33,Росэксимбанк,8814
...,...,...
348,«ИНЭКО»,5011664
349,ВБРР,16995
350,ПСБ,5306
351,СМП Банк,17200


In [176]:
from parcer_functions import Download_bank_pages
cls_parcer = Download_bank_pages(year = 2023, bank_ids_table=problem_ids_df, no_fl_assets=True)

In [177]:
cls_parcer.parce_year_table()

bank row downloaded 1
bank row downloaded 2
bank row downloaded 3
bank row downloaded 4
bank row downloaded 5
bank row downloaded 6
bank row downloaded 7
bank row downloaded 8
bank row downloaded 9
bank row downloaded 10
bank row downloaded 11
bank row downloaded 12
bank row downloaded 13
bank row downloaded 14
bank row downloaded 15
bank row downloaded 16
bank row downloaded 17
БНП Париба Банк https://www.banki.ru/banks/ratings/?BANK_ID=17151&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2023-12-01&date2=2023-01-01
bank row downloaded 18
bank row downloaded 19
bank row downloaded 20
bank row downloaded 21
bank row downloaded 22
bank row downloaded 23
bank row downloaded 24
bank row downloaded 25
bank row downloaded 26
Балтинвестбанк https://www.banki.ru/banks/ratings/?BANK_ID=68717&IS_SHOW_GROUP=0&IS_SHOW_LIABILITIES=0&date1=2023-12-01&date2=2023-01-01
bank row downloaded 27
bank row downloaded 28
«Натиксис Банк» https://www.banki.ru/banks/ratings/?BANK_ID=17276&IS_SHOW_GROUP=0&IS_SHOW_

In [178]:
fin_df1 = cls_parcer.final_df

In [181]:
fin_df1.columns.name = 0
fin_df1 = fin_df1.reset_index().drop(columns = 'index')
df_for_save1 = fin_df1[['bank_name','Вложения в ценные бумаги', 'Кредитный портфель',
       'Просроченная задолженность в кредитном портфеле', 'Активы нетто',
       'Векселя', 'Вклады физических лиц', 'Вложения в акции',
       'Вложения в векселя', 'Вложения в капиталы других организаций',
       'Вложения в облигации', 'Выданные МБК',
       'Выпущенные облигации и векселя', 'Высоколиквидные активы',
       'Капитал (по форме 123)', 'Кредиты предприятиям и организациям',
       'Кредиты физическим лицам', 'ЛОРО-счета', 'НОСТРО-счета', 'Облигации',
       'Основные средства и нематериальные активы', 'Привлеченные МБК',
       'Привлеченные от ЦБ РФ', 'Прочие активы', 'Размещенные МБК в ЦБ РФ',
       'Средства предприятий и организаций', 'ФЛ Счета', 'Чистая прибыль',
       'Н1', 'Н2', 'Н3',
       'Уровень просроченной задолженности по кредитному портфелю']]

In [124]:
df_for_save1 = cls_parcer.prettify_final_df(fin_df1)

In [183]:
df_for_save1.to_excel('chasti/data_2023_problem.xlsx', index=False)

In [185]:
data_2023 = pd.read_excel('chasti/data_2023.xlsx')
data_2023_problem = pd.read_excel('chasti/data_2023_problem.xlsx')

In [186]:
total_2023 = pd.concat([data_2023, data_2023_problem])

In [188]:
total_2023.to_excel('full_dfs/df_2023_full.xlsx', index=False)

In [187]:
total_2023

,bank_name,Вложения в ценные бумаги,Кредитный портфель,Просроченная задолженность в кредитном портфеле,Активы нетто,Векселя,Вклады физических лиц,Вложения в акции,Вложения в векселя,Вложения в капиталы других организаций,...,Привлеченные от ЦБ РФ,Прочие активы,Размещенные МБК в ЦБ РФ,Средства предприятий и организаций,ФЛ Счета,Чистая прибыль,Н1,Н2,Н3,Уровень просроченной задолженности по кредитному портфелю
0,Сбербанк,6.292502e+09,3.656594e+10,155268282.0,50695718774,77202098,11458178474,45827036,580517,1270984158,...,1070440895,2036300583,0,13751643094,3645562530,1377559523,13.01,64.92,86.28,0.43
1,ВТБ,4.693619e+09,1.573751e+10,135018281.0,26188218945,44638934,4579261369,83808718,0,1072255743,...,1395688627,1882539515,0,11575806309,1925335464,185532759,9.46,60.91,80.50,0.86
2,Газпромбанк,1.471975e+09,1.024813e+10,58494498.0,15296029418,106779045,1592032837,26906711,0,541594156,...,479418376,650944656,0,7964930669,2538942392,239160122,11.02,90.13,66.72,0.57
3,Альфа-Банк,1.072772e+09,5.716990e+09,23056930.0,8411400483,22954997,1383959353,28971157,1000,15415862,...,138860185,472392973,0,3376395087,1492431122,99718083,11.91,95.42,77.48,0.42
4,Московский кредитный банк (МКБ),8.645898e+08,3.129535e+09,29047964.0,4971033526,0,514246382,20754861,0,8056345,...,60881212,205249448,60000000,2298140312,277308455,41144717,12.07,90.83,66.29,0.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53,АЗИЯПЭЙ,NaN,NaN,NaN,189126,0,0,0,0,0,...,0,515,105600,0,0,-3041,516.81,0.00,0.00,0.00
54,Промсвязьинвест,NaN,NaN,NaN,185626,0,0,0,0,0,...,0,6773,156000,62654,62654,7224,279.82,0.00,0.00,0.00
55,«Петербургский Расчетный Центр»,2.080000e+02,NaN,NaN,167242,0,0,208,0,45,...,0,3631,159181,1105,1105,18664,298.26,0.00,0.00,0.00
56,«Дальний Восток»,NaN,NaN,NaN,155969,0,0,0,0,0,...,0,16209,91000,15889,15889,10669,95.16,0.00,0.00,0.00
